### Data Clean

In [23]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [24]:
df = pd.read_csv(r'G:\My Drive\Creative_Meta_Analysis\Data\KMB Essence Google Meta Output Wave 1_FINAL_REVISED - 2.6.18.csv',
                 encoding='latin1', 
                 na_values='', 
                 keep_default_na=False)

c:\users\william.raikes\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,SID,Country,Region,Test Start Date,Test End Date,Quarter Year,Campaign,ID,Age 1,Age 2,Age 3,Gender,Operating System,Unaided Awareness,Aided Awareness,(Purchase) Intent,Consideration Intent,Enjoyment,Descriptors 1,Descriptors 2,Descriptors 3,Understanding,Relevance,Branding,Brand Appeal,Distinctiveness,Set Cell,Creative Name,Format,Publisher,Platform,Product,Video Length
0,259311,US,NA,10/30/2015,11/4/2015,Q4 2015,Essence Baldwin US 2015,147,2.0,NaN,NaN,1.0,NaN,test test test test test test test test test t...,1.0,4.0,NaN,2.0,0.0,0.0,1.0,NaN,NaN,2.0,2.0,1.0,2.0,Questions (259311B),Video,YouTube,Desktop,GSA,NaN
1,259311,US,NA,10/30/2015,11/4/2015,Q4 2015,Essence Baldwin US 2015,153,1.0,NaN,NaN,1.0,NaN,"google, bing, yahoo",1.0,2.0,NaN,2.0,0.0,0.0,1.0,NaN,NaN,0.0,1.0,1.0,2.0,Questions (259311B),Video,YouTube,Desktop,GSA,NaN
2,259311,US,NA,10/30/2015,11/4/2015,Q4 2015,Essence Baldwin US 2015,156,3.0,NaN,NaN,1.0,NaN,google,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,American Red Cross (259311D),Video,YouTube,Desktop,GSA,NaN
3,259311,US,NA,10/30/2015,11/4/2015,Q4 2015,Essence Baldwin US 2015,159,2.0,NaN,NaN,1.0,NaN,Google,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,American Red Cross (259311D),Video,YouTube,Desktop,GSA,NaN
4,259311,US,NA,10/30/2015,11/4/2015,Q4 2015,Essence Baldwin US 2015,160,2.0,NaN,NaN,1.0,NaN,yahoo,1.0,1.0,NaN,2.0,3.0,3.0,3.0,NaN,NaN,4.0,2.0,2.0,1.0,Windy (259311A),Video,YouTube,Desktop,GSA,NaN


In [4]:
#Recode 'Age 1' NaN if 'Age 2' == 0.
df['Age 1'] = df.apply(lambda x: 1 if x['Age 2'] == 0 else x['Age 1'], axis=1)

In [5]:
#Remove spaces and parentheses in column names.
new_cols = []

for col in df.columns:
    new_col = col
    
    new_col = new_col.replace(' ', '_')
    new_col = new_col.replace('(', '')
    new_col = new_col.replace(')', '')
    
    new_cols.append(new_col)
    
#df.columns = new_cols
df.columns = new_cols

In [6]:
df.rename(
    columns={"Descriptors_1": "Interesting", 
             "Descriptors_2": "Unique", 
             "Descriptors_3": "Involving"}, 
    inplace=True)

In [7]:
#Recode 'Age 1' into 'Age_Recode' based on below conditionals.
def age_recode(row):
    if np.isnan(row):
        return row
    elif np.isin(row, [1, 2]):
        return '18-29'
    elif np.isin(row, [3, 4, 5]):
        return '30-44'
    else:
        return '45+'

df['Age_Recode'] = df.apply(lambda x: age_recode(x['Age_1']), axis=1)

In [8]:
df.drop(columns=[
    'SID', 
    'ID',
    'Age_1',
    'Age_2', 
    'Age_3', 
    'Test_Start_Date', 
    'Test_End_Date', 
    'Unaided_Awareness'
    ], inplace=True
)

In [9]:
#Recode ordinal data into binary top 2 box.
def top_box_recode(row):
    if np.isnan(row):
        return row
    elif np.isin(row, [0, 1]):
        return 1
    else:
        return 0

In [10]:
labels = [
    'Purchase_Intent',
    'Enjoyment',
    'Interesting',
    'Unique',
    'Involving',
    'Understanding',
    'Relevance',
    'Branding',
    'Brand_Appeal',
    'Distinctiveness'
]

for label in labels:
    df[label + '_Top2Box'] = df.apply(lambda x: top_box_recode(x[label]), axis=1)
    
for label in labels:
    df[label] = df[label].max() - df[label]

In [11]:
df.replace(
    {
        'Operating_System': {0: 'Android', 1: 'iOS'},
        'Gender': {0: 'Male', 1: 'Female'}
    }, inplace=True
)

In [13]:
df = df[[
    'Country', 'Region', 'Quarter_Year', 'Campaign', 'Gender', 'Age_Recode',
    'Operating_System', 'Aided_Awareness', 'Purchase_Intent',
    'Purchase_Intent_Top2Box', 'Consideration_Intent', 'Enjoyment', 
    'Enjoyment_Top2Box', 'Interesting', 'Interesting_Top2Box',
    'Unique', 'Unique_Top2Box', 'Involving', 'Involving_Top2Box',
    'Understanding', 'Understanding_Top2Box', 'Relevance', 
    'Relevance_Top2Box', 'Branding', 'Branding_Top2Box', 'Brand_Appeal',
    'Brand_Appeal_Top2Box', 'Distinctiveness', 'Distinctiveness_Top2Box',
    'Set_Cell', 'Creative_Name', 'Format', 'Publisher',
    'Platform', 'Product', 'Video_Length'
]]

In [14]:
df.head(20)

,Country,Region,Quarter_Year,Campaign,Gender,Age_Recode,Operating_System,Aided_Awareness,Purchase_Intent,Purchase_Intent_Top2Box,Consideration_Intent,Enjoyment,Enjoyment_Top2Box,Interesting,Interesting_Top2Box,Unique,Unique_Top2Box,Involving,Involving_Top2Box,Understanding,Understanding_Top2Box,Relevance,Relevance_Top2Box,Branding,Branding_Top2Box,Brand_Appeal,Brand_Appeal_Top2Box,Distinctiveness,Distinctiveness_Top2Box,Set_Cell,Creative_Name,Format,Publisher,Platform,Product,Video_Length
0,US,NA,Q4 2015,Essence Baldwin US 2015,Female,18-29,NaN,1.0,1.0,0.0,NaN,2.0,0.0,3.0,1.0,3.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,2.0,1.0,2.0,Questions (259311B),Video,YouTube,Desktop,GSA,NaN
1,US,NA,Q4 2015,Essence Baldwin US 2015,Female,18-29,NaN,1.0,3.0,0.0,NaN,2.0,0.0,3.0,1.0,3.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,4.0,1.0,3.0,1.0,2.0,1.0,2.0,Questions (259311B),Video,YouTube,Desktop,GSA,NaN
2,US,NA,Q4 2015,Essence Baldwin US 2015,Female,30-44,NaN,1.0,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,American Red Cross (259311D),Video,YouTube,Desktop,GSA,NaN
3,US,NA,Q4 2015,Essence Baldwin US 2015,Female,18-29,NaN,0.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,American Red Cross (259311D),Video,YouTube,Desktop,GSA,NaN
4,US,NA,Q4 2015,Essence Baldwin US 2015,Female,18-29,NaN,1.0,4.0,1.0,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,2.0,0.0,1.0,0.0,1.0,Windy (259311A),Video,YouTube,Desktop,GSA,NaN
5,US,NA,Q4 2015,Essence Baldwin US 2015,Female,18-29,NaN,1.0,5.0,1.0,NaN,1.0,0.0,3.0,1.0,3.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,2.0,0.0,1.0,0.0,2.0,Questions (259311B),Video,YouTube,Desktop,GSA,NaN
6,US,NA,Q4 2015,Essence Baldwin US 2015,Female,18-29,NaN,1.0,5.0,1.0,NaN,3.0,1.0,3.0,1.0,3.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,3.0,1.0,3.0,1.0,2.0,1.0,2.0,Questions (259311B),Video,YouTube,Desktop,GSA,NaN
7,US,NA,Q4 2015,Essence Baldwin US 2015,Male,30-44,NaN,1.0,4.0,1.0,NaN,3.0,1.0,3.0,1.0,3.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,3.0,1.0,3.0,1.0,2.0,1.0,3.0,Hot Air Balloon (259311C),Video,YouTube,Desktop,GSA,NaN
8,US,NA,Q4 2015,Essence Baldwin US 2015,Male,18-29,NaN,0.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,American Red Cross (259311D),Video,YouTube,Desktop,GSA,NaN
9,US,NA,Q4 2015,Essence Baldwin US 2015,Female,18-29,NaN,1.0,4.0,1.0,NaN,2.0,0.0,2.0,1.0,2.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,3.0,1.0,2.0,0.0,1.0,0.0,2.0,Questions (259311B),Video,YouTube,Desktop,GSA,NaN


In [15]:
df.to_csv(r'G:\My Drive\Creative_Meta_Analysis\Data\Clean_Data.csv', index=False)